In [13]:
import csv

# Mock dataset (unchanged)
sample_data = [
    {"Campaign_ID": "C001", "Segment": "Electronics", "Clicks": 250, "Conversions": 30},
    {"Campaign_ID": "C002", "Segment": "Electronics", "Clicks": 180, "Conversions": 22},
    {"Campaign_ID": "C003", "Segment": "Fitness", "Clicks": 140, "Conversions": 18},
    {"Campaign_ID": "C004", "Segment": "Fashion", "Clicks": 100, "Conversions": 12},
    {"Campaign_ID": "C005", "Segment": "Electronics", "Clicks": 90, "Conversions": 10},
    {"Campaign_ID": "C006", "Segment": "Fitness", "Clicks": 70, "Conversions": 8}
]

# Keyword mapping (unchanged)
keyword_map = {
    "Electronics": ["samsung galaxy watch", "apple iphone charger", "samsung wireless earbuds"],
    "Fitness": ["nike running shoes", "fitbit fitness tracker"],
    "Fashion": ["adidas sneakers", "levis denim jacket"]
}

# Helper function (unchanged)
def is_compatible_keyword(title_keyword, desc_keyword):
    brands = ["samsung", "apple", "sony", "nike", "fitbit", "adidas", "levis"]
    title_brand = next((b for b in brands if b in title_keyword.lower()), None)
    desc_brand = next((b for b in brands if b in desc_keyword.lower()), None)
    return not title_brand or (desc_brand == title_brand or not desc_brand)

# Solution 1: Keyword Extraction & Lemmatization (Unchanged)
def solution1_extract_reorder(data):
    ranked_keywords = []
    for item in data:
        segment = item["Segment"]
        keyword_idx = {"C001": 0, "C002": 1, "C005": 2}.get(item["Campaign_ID"], 0)
        keyword = keyword_map[segment][min(keyword_idx, len(keyword_map[segment])-1)]
        score = item["Clicks"] * 0.7 + item["Conversions"] * 0.3
        lemma = keyword.replace("shoes", "shoe")
        ranked_keywords.append((lemma, score, segment))
    ranked_keywords.sort(key=lambda x: x[1], reverse=True)

    top_keyword, _, top_segment = ranked_keywords[0]
    title = top_keyword.title()
    segment_keywords = [kw[0] for kw in ranked_keywords if kw[2] == top_segment and kw[0] != top_keyword]
    compatible_keywords = [kw for kw in segment_keywords if is_compatible_keyword(top_keyword, kw)]
    if not compatible_keywords:
        compatible_keywords = ["accessories"]

    description = f"Enjoy seamless pairing with our premium {compatible_keywords[0]} for top performance and style."
    return {"title": title, "description": description.strip()}

# Solution 2: NER-Enhanced (Updated)
def solution2_ner_enhanced(data):
    ranked_keywords = []
    brand_terms = ["samsung", "apple", "sony", "nike", "fitbit", "adidas", "levis"]
    for item in data:
        segment = item["Segment"]
        keyword_idx = {"C001": 0, "C002": 1, "C005": 2}.get(item["Campaign_ID"], 0)
        keyword = keyword_map[segment][min(keyword_idx, len(keyword_map[segment])-1)]
        score = item["Clicks"] * 0.7 + item["Conversions"] * 0.3
        lemma = keyword.replace("shoes", "shoe")
        # Increased NER boost to 1.5x
        final_score = score * 1.5 if any(b in lemma for b in brand_terms) else score
        ranked_keywords.append((lemma, final_score, segment))
    ranked_keywords.sort(key=lambda x: x[1], reverse=True)

    top_keyword, _, top_segment = ranked_keywords[0]
    title = top_keyword.title()
    segment_keywords = [kw[0] for kw in ranked_keywords if kw[2] == top_segment and kw[0] != top_keyword]
    compatible_keywords = [kw for kw in segment_keywords if is_compatible_keyword(top_keyword, kw)]
    if not compatible_keywords:
        compatible_keywords = ["accessories"]

    # Updated description to highlight NER boost
    description = f"Optimized for use with our premium {compatible_keywords[0]} thanks to advanced recognition technology."
    return {"title": title, "description": description.strip()}

# Execute both solutions
output1 = solution1_extract_reorder(sample_data)
output2 = solution2_ner_enhanced(sample_data)

# Save to CSV
with open("optimized_listing.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["Solution", "Title", "Description"])
    writer.writerow(["Keyword + Lemmatization", output1["title"], output1["description"]])
    writer.writerow(["NER Enhanced", output2["title"], output2["description"]])

# Display results
print("Solution 1 (Keyword + Lemmatization):")
print(f"  Title: {output1['title']}")
print(f"  Description: {output1['description']}")
print("\nSolution 2 (NER Enhanced):")
print(f"  Title: {output2['title']}")
print(f"  Description: {output2['description']}")

Solution 1 (Keyword + Lemmatization):
  Title: Samsung Galaxy Watch
  Description: Enjoy seamless pairing with our premium samsung wireless earbuds for top performance and style.

Solution 2 (NER Enhanced):
  Title: Samsung Galaxy Watch
  Description: Optimized for use with our premium samsung wireless earbuds thanks to advanced recognition technology.
